In [49]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import pandas as pd
import plotly.graph_objects as go

In [7]:
dataFrame = pd.read_table(r"./Data/FDAXH22-EUREX.scid_BarData.txt", sep = ", ", engine = 'python')

In [8]:
Date, Time, Open, High, Low = dataFrame.Date, dataFrame.Time, dataFrame.Open, dataFrame.High, dataFrame.Low
Last, Volume, NumberOfTrades =  dataFrame.Last, dataFrame.Volume, dataFrame.NumberOfTrades
BidVolume, AskVolume = dataFrame.BidVolume, dataFrame.AskVolume

In [9]:
Spread = [High[i]-Low[i] for i in range(len(High))]
Delta = [AskVolume[i] - BidVolume[i] for i in range(len(BidVolume))]

CumulativeDelta = [Delta[0]]
for i in range(1, len(Delta)):
    nouvelle_valeur = CumulativeDelta[-1] + Delta[i]
    CumulativeDelta.append(nouvelle_valeur)

In [20]:
def moyenne_mobile(k,n):
    if k<23:
        sum_= 0
        for i in range(k+1):
            sum_ += Last[i]**n
        return sum_/(k+1)
    else:
        sum_ = 0
        for i in range(k, k-23, -1):
            sum_ += Last[i]**n
        return sum_/23
    
def variance_mobile(k):
    return moyenne_mobile(k,2)-(moyenne_mobile(k,1))**2

In [21]:
def variance_mobile(k):
    return moyenne_mobile(k,2)-(moyenne_mobile(k,1))**2

In [22]:
Moy_mobile = [moyenne_mobile(k,1) for k in range(len(Last))]

In [23]:
Var_mobile = [variance_mobile(k) for k in range(len(Last))]

In [24]:
z_score = []
def z_scoring():
    for k in range(len(Last)):
        if Var_mobile[k] != 0:
            z_score.append(Last[k] - Moy_mobile[k]/Var_mobile[k])
        else:
            z_score.append(0)

In [25]:
z_scoring()

In [10]:
import datetime
from datetime import datetime

jours_semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
weekdays = [jours_semaine[datetime.strptime(dataFrame.Date[i][2:], '%y/%m/%d').weekday()] for i in range(len(dataFrame))]

In [27]:
file = open(r"./Data/FDAXH22-Full.txt", "w+")
file.write("Date, Time, Open, High, Low, Last, Volume, NumberOfTrades, BidVolume, AskVolume, Delta, Cumulative Delta, Spread, Moyenne Mobile, Z Score, Jour\n")
for i in range(len(Date)):
    line = str(Date[i]) + ", " + str(Time[i]) + ", " + str(Open[i]) + ", " + str(High[i]) + ", " + str(Low[i]) + ", " 
    line+= str(Last[i]) + ", " + str(Volume[i]) + ", " + str(NumberOfTrades[i]) + ", " + str(BidVolume[i]) + ", " 
    line+= str(AskVolume[i]) + ", " + str(Delta[i]) + ", " + str(CumulativeDelta[i]) +", " + str(Spread[i]) + ", " 
    line+= str(Moy_mobile[i]) + ", " + str(z_score[i]) + ", " + str(weekdays[i]) + "\n"
    file.write(line)
    
file.close()

In [ ]:
#on insere les nouvelles colonnes dans le dataFrame 

dataFrame.insert(loc=len(dataFrame.columns), column='Delta', value=pd.Series(Delta))
dataFrame.insert(loc=len(dataFrame.columns), column='Cumulative Delta', value=pd.Series(CumulativeDelta))
dataFrame.insert(loc=len(dataFrame.columns), column='Spread', value=pd.Series(Spread))
dataFrame.insert(loc=len(dataFrame.columns), column='Moyenne Mobile (23)', value=pd.Series(Moy_mobile))
dataFrame.insert(loc=len(dataFrame.columns), column='Jour', value=pd.Series(weekdays))

## Création d'un DataFrame journalier

In [26]:
# requete SQL pour recuperer toutes les entrees ou l'heure est egale à 8h du matin
# permet de recuperer la liste des jours, et la liste des prix d'ouverture journaliers
daily_morning = dataFrame.loc[dataFrame['Time'] == '08:00:00']
days = list(daily_morning.Date)
daily_open = list(daily_morning.Open)

In [27]:
# pour chaque jour, on fait une requete pour selectionner la ligne ou l'heure la derniere de la journee
# permet de recuperer la liste des fermtures journalieres

daily_last = []
for i in range(len(days)-1): #le dernier jour ne s'arrete pas a 22h, d'ou le -1 dans le range
    df_last_minute = dataFrame.loc[(dataFrame.Time == '22:05:00') & (dataFrame.Date == days[i])]
    if len(df_last_minute) == 0:
        df_last_minute = dataFrame.loc[(dataFrame.Time == '22:00:00') & (dataFrame.Date == days[i])]
    if len(df_last_minute) == 0:
        df_last_minute = dataFrame.loc[(dataFrame.Time == '21:55:00') & (dataFrame.Date == days[i])]
    if len(df_last_minute) == 0:
        df_last_minute = dataFrame.loc[(dataFrame.Time == '21:50:00') & (dataFrame.Date == days[i])]
    cur_last = list(df_last_minute.Last)
    daily_last.append(int(cur_last[0]))
    
daily_last.append(Last[len(Last)-1]) #le dernier jour ne s'arrete pas a 22h

In [28]:
# on fait des requetes pour recuperer toutes les donnees d'une journee
# cela permet de calculer le high de la journee, le low de la journee et le volume total de la journee
# on repete cela pour tous les jours
# relativement long en temps d'exécution mais permet de s'assurer de prendre toutes les entrees d'une journee
# par ex, si une journee se termine a 22h05 et non 22h, cette technique permet de s'assurer de bien prendre toutes les tranches

daily_high = []
daily_low = []
daily_volume = []

for day in days:
    df_day = dataFrame.loc[dataFrame.Date == day]
    
    volume_journalier = np.sum(list(df_day.Volume))
    daily_volume.append(volume_journalier)
    
    high_journalier = max(list(df_day.High))
    daily_high.append(high_journalier)
    
    low_journalier = min(list(df_day.Low))
    daily_low.append(low_journalier)

In [29]:
dataFrame_daily = pd.DataFrame()
dataFrame_daily.insert(loc = 0, column='Date', value=pd.Series(days))
dataFrame_daily.insert(loc = 1, column='Open', value=pd.Series(daily_open))
dataFrame_daily.insert(loc = 2, column='High', value=pd.Series(daily_high))
dataFrame_daily.insert(loc = 3, column='Low', value=pd.Series(daily_low))
dataFrame_daily.insert(loc = 4, column='Close', value=pd.Series(daily_last))
dataFrame_daily.insert(loc = 5, column='Volume', value=pd.Series(daily_volume))

## Pourcentage de High/Low matinal égal au High/Low journalier

In [30]:
opening_spread_max = []
opening_spread_min = []
times_matin = ['08:00:00', '08:05:00', '08:10:00', '08:15:00', '08:20:00', '08:25:00', '08:30:00', '08:35:00', '08:40:00', '08:45:00', '08:50:00', '08:55:00']
def opening_spread():
    for day in days:
        maxi = int(dataFrame.loc[(dataFrame.Time == times_matin[0]) & (dataFrame.Date == day)].High) 
        mini = int(dataFrame.loc[(dataFrame.Time == times_matin[0]) & (dataFrame.Date == day)].Low)
        for i in range(1,12):
            df_minute = dataFrame.loc[(dataFrame.Time == times_matin[i]) & (dataFrame.Date == day)]
            if len(df_minute) !=0 :
                m = int(df_minute.High)
                l = int(df_minute.Low)
                if m > maxi: 
                    maxi = m
                if l < mini:
                    mini = l
        opening_spread_max.append(maxi)
        opening_spread_min.append(mini)

In [31]:
def correspondances_opening_day_spread2():
    jours_max_correspondants = []
    jours_min_correspondants = []
    
    nb_jours = 0
    for day in days:
        dataFrame_day = dataFrame_daily.loc[dataFrame_daily.Date == day]
        if abs(int(dataFrame_day.High) - opening_spread_max[nb_jours]) <= 15:
            jours_max_correspondants.append(day)
        if abs(int(dataFrame_day.Low) - opening_spread_min[nb_jours]) <= 15:
            jours_min_correspondants.append(day)
        nb_jours += 1
    return jours_max_correspondants, jours_min_correspondants

In [32]:
opening_spread()

In [33]:
jours_max_correspondants, jours_min_correspondants = correspondances_opening_day_spread2()
print("Nombre de jours où le max entre 8h et 9h est le max de la journée :", len(jours_max_correspondants))
print("Nombre de jours où le min entre 8h et 9h est le min de la journée :", len(jours_min_correspondants))
print("La combinaison de ces deux cas représente", (len(jours_max_correspondants) + len(jours_min_correspondants))/len(days), "% du temps")

Nombre de jours où le max entre 8h et 9h est le max de la journée : 735
Nombre de jours où le min entre 8h et 9h est le min de la journée : 685
La combinaison de ces deux cas représente 0.6187363834422658 % du temps


In [35]:
times_morning = ['08:00:00', '08:05:00', '08:10:00', '08:15:00', '08:20:00', '08:25:00', '08:30:00', 
                '08:35:00', '08:40:00', '08:45:00', '08:50:00', '08:55:00']
daily_morning_last = []
for i in range(len(days)): 
    k = len(times_morning) - 1
    while len(dataFrame.loc[(dataFrame.Time == times_morning[k]) & (dataFrame.Date == days[i])]) <= 0:
        k-=1
    df_last_minute = dataFrame.loc[(dataFrame.Time == times_morning[k]) & (dataFrame.Date == days[i])]
    cur_last = list(df_last_minute.Last)
    daily_morning_last.append(int(cur_last[0]))

In [41]:
import datetime
from datetime import datetime

jours_semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
jours = [jours_semaine[datetime.strptime(day[2:], '%y/%m/%d').weekday()] for day in days]

In [48]:
file = open(r"./Data/FDAXH22-matinal.txt", "w+")
file.write("Date,Jour,Open Matinal,High Matinal,Low Matinal,Last Matinal\n")
for i in range(len(days)):
    line = days[i] + "," + str(jours[i]) + "," + str(daily_open[i]) + "," + str(opening_spread_max[i]) + "," + str(opening_spread_min[i]) + "," + str(daily_morning_last[i]) +"\n"
    file.write(line)
    
file.close()